# <center>Big Data &ndash; Moodle Exercises </center>
## <center>Fall 2024 &ndash; Week 9 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and Spark SQL, Moodle exercise</center>

# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week. It will be the same language game dataset as seen in the exercises. If you still do not have created the `confusion-part.json` file, follow these instruction. If you only have it in the `exercise08` folder please copy the `confusion-2014-03-02` folder in the `notebooks` folder of your exam magic box.

1. Move to the `notebooks` folder in the terminal
2. Download the data: <br>
   ```wget https://f003.backblazeb2.com/file/larsyencken-eu-public/greatlanguagegame/confusion-2014-03-02.tbz2``` <br>
   __or__ <br>
   ```curl -O https://f003.backblazeb2.com/file/larsyencken-eu-public/greatlanguagegame/confusion-2014-03-02.tbz2```
3. Extract the data: <br>
   ```tar -jxvf confusion-2014-03-02.tbz2```
4. Change directory to ```confusion-2014-03-02```
5. Extract the part of the dataset that we will work with in this exercise: <br>
   ```head -n 3000000 confusion-2014-03-02.json > confusion-part.json```
## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in the `README.md` file inside the data bundle.

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query as well as the productivity time. That means the development time of each query (time elapsed from before you start writing the query, until the time at which the correct, final query is ready). Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

In [1]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise():
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))

## <center>1. Spark Dataframes</center>

Write queries for the same dataset as last week, but this time using Spark Dataframes operations (the data loading will take a couple minutes)

### 1.0. Data preprocessing

In [2]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02/confusion-part.json"
dataset = spark.read.json(path).cache()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/21 01:18:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#test it out
dataset.limit(3).show()

+--------------------+-------+----------+---------+--------------------+---------+
|             choices|country|      date|    guess|              sample|   target|
+--------------------+-------+----------+---------+--------------------+---------+
|[Maori, Mandarin,...|     AU|2013-08-19|Norwegian|48f9c924e0d98c959...|Norwegian|
|[Danish, Dinka, K...|     AU|2013-08-19|    Dinka|af5e8f27cef9e689a...|    Dinka|
|[German, Hungaria...|     AU|2013-08-19|  Turkish|509c36eb58dbce009...|   Samoan|
+--------------------+-------+----------+---------+--------------------+---------+



## Assignment 1
Find the number of games where one of the choices is Italian.

**confused**: why is this wrong?
How do I filter correctly

In [4]:
start_exercise()

In [5]:
from pyspark.sql.functions import explode

exploded = dataset.select("sample", explode("choices").alias("choice"))
exploded.filter(exploded["choice"] == "Italian").select("sample").distinct().count()

399

In [8]:
from pyspark.sql.functions import array_contains

dataset.filter(array_contains("choices", "Italian")).select("sample").distinct().count()

399

In [9]:
finish_exercise()

This exercise took 359s


## Assignment 2
Return the number of (distinct) countries.

In [10]:
start_exercise()

In [11]:
dataset.select("country").distinct().count()

188

In [12]:
finish_exercise()

This exercise took 25s


## Assignment 3
Return the sample IDs (i.e., the "sample" field) of the top 2 games played in Switzerland (CH) where the guessed language is correct (equal to the target one) ordered by date (ascending), then by language (descending).

In [13]:
start_exercise()

In [14]:
from pyspark.sql.functions import asc, desc

dataset.filter(dataset["guess"] == dataset["target"]) \
    .filter(dataset["country"] == "CH") \
    .orderBy(asc("date"), desc("target")) \
    .select("sample") \
    .limit(2) \
    .show(truncate=False)

+--------------------------------+
|sample                          |
+--------------------------------+
|74b5340a230b1e0c1d45787bc4280b05|
|b7df3f9d67cef259fbcaa5abcad9d774|
+--------------------------------+



In [15]:
finish_exercise()

This exercise took 185s


## Assignment 4
Aggregate all games by guess and target language, ignoring games with correct guesses, count the number of guesses for each group and return the pair with the highest count i.e. the most confused pair.

In [16]:
start_exercise()

In [56]:
from pyspark.sql.functions import asc, desc

dataset.filter(dataset["target"] != dataset["guess"]) \
    .select("target", "guess") \
    .groupBy("target", "guess") \
    .count() \
    .withColumnRenamed("count", "guesses") \
    .orderBy(desc("guesses")) \
    .select("guess", "target") \
    .limit(1) \
    .show(truncate=False)

+--------+---------+
|guess   |target   |
+--------+---------+
|Mandarin|Cantonese|
+--------+---------+



In [21]:
finish_exercise()

This exercise took 308s


## Assignment 5
Find the language with the second highest guessed percentage in Switzerland.

**confused**: why is this wrong? Was it asking for most overall guessed, or most correctly guessed?

In [22]:
start_exercise()

In [28]:
from pyspark.sql.functions import asc, desc

dataset.filter(dataset["country"] == "CH") \
    .select("guess") \
    .groupBy("guess") \
    .count() \
    .orderBy(desc("count")) \
    .select("guess") \
    .limit(2) \
    .show(truncate=False)

+-------+
|guess  |
+-------+
|Russian|
|Italian|
+-------+



In [29]:
finish_exercise()

This exercise took 218s


## Assignment 6
On which day was any sample played the most ever? Hint: you might need to import `max()`.

In [30]:
start_exercise()

In [67]:
from pyspark.sql.functions import max, desc

dataset.select("sample", "date") \
    .groupBy("date") \
    .count() \
    .withColumnRenamed("count", "samples") \
    .orderBy(desc("samples")) \
    .select("date") \
    .limit(1) \
    .show(truncate=False)

+----------+
|date      |
+----------+
|2013-09-04|
+----------+



In [33]:
finish_exercise()

This exercise took 177s


## <center>2. Spark SQL</center>

Now, for the second weekly quiz, write Spark SQL queries for the same questions as earlier.

### 2.0. Data preprocessing

In [34]:
!pip install sparksql-magic --quiet

In [35]:
%load_ext sparksql_magic

In [36]:
path = "confusion-2014-03-02/confusion-part.json"
dataset = spark.read.json(path).cache()
dataset.createOrReplaceTempView("dataset")

25/01/21 01:40:37 WARN CacheManager: Asked to cache already cached data.        


In [37]:
%%sparksql
-- test it out
SELECT *
FROM dataset
LIMIT 3

choices,country,date,guess,sample,target
"['Maori', 'Mandarin', 'Norwegian', 'Tongan']",AU,2013-08-19,Norwegian,48f9c924e0d98c959d8a6f1862b3ce9a,Norwegian
"['Danish', 'Dinka', 'Khmer', 'Lao']",AU,2013-08-19,Dinka,af5e8f27cef9e689a070b8814dcc02c3,Dinka
"['German', 'Hungarian', 'Samoan', 'Turkish']",AU,2013-08-19,Turkish,509c36eb58dbce009ccf93f375358d53,Samoan


## Assignment 1
Find the number of games where one of the choices is Italian.

**confused**: why does putting a distinct here mess up the query? I am making the samples distinct, right?

In [38]:
start_exercise()

In [39]:
%%sparksql

SELECT COUNT(DISTINCT(sample))
FROM dataset
WHERE array_contains(choices, 'Italian')

count(DISTINCT sample)
399


In [40]:
%%sparksql

SELECT COUNT(DISTINCT(sample))
FROM dataset
LATERAL VIEW EXPLODE(choices) as choice
WHERE choice == 'Italian'

count(DISTINCT sample)
399


In [41]:
finish_exercise()

This exercise took 167s


## Assignment 2
Return the number of (distinct) countries.

In [42]:
start_exercise()

In [45]:
%%sparksql

SELECT COUNT(DISTINCT(country))
FROM dataset

count(DISTINCT country)
188


In [46]:
finish_exercise()

This exercise took 63s


## Assignment 3
Return the sample IDs (i.e., the "sample" field) of the top 2 games played in Switzerland (CH) where the guessed language is correct (equal to the target one) ordered by date (ascending), then by language (descending).

In [47]:
start_exercise()

In [49]:
%%sparksql

SELECT sample
FROM dataset
WHERE guess = target
AND country = 'CH'
ORDER BY date ASC, target DESC
LIMIT 2

sample
74b5340a230b1e0c1d45787bc4280b05
b7df3f9d67cef259fbcaa5abcad9d774


In [50]:
finish_exercise()

This exercise took 66s


## Assignment 4
Aggregate all games by guess and target language, ignoring games with correct guesses, count the number of guesses for each group and return the pair with the highest count i.e. the most confused pair.

In [51]:
start_exercise()

In [54]:
%%sparksql

SELECT guess, target
FROM dataset
WHERE guess != target
GROUP BY guess, target
ORDER BY COUNT(*) DESC
LIMIT 1

guess,target
Mandarin,Cantonese


In [55]:
finish_exercise()

This exercise took 111s


## Assignment 5
Find the language with the second highest guessed percentage in Switzerland.

**confused**: Did they mean the most guessed language overall, or the most correctly guessed language?

In [57]:
start_exercise()

In [62]:
%%sparksql
SELECT guess
FROM dataset
WHERE country == 'CH'
GROUP BY guess
ORDER BY COUNT(*) DESC
LIMIT 2

guess
Russian
Italian


In [63]:
finish_exercise()

This exercise took 129s


## Assignment 6
On which day was any sample played the most ever?

In [64]:
start_exercise()

In [65]:
%%sparksql
SELECT date
FROM dataset
GROUP BY date
ORDER BY COUNT(sample) DESC
LIMIT 1

date
2013-09-04


In [66]:
finish_exercise()

This exercise took 76s
